In [63]:
import paper_analyzer as pa
import re

search = pa.Search(query="all:electron", start=0, max_results=10)

# for paper in search.results():
#     print(paper)
#     print("")
    
df = search.to_dataframe()
df

,paper_id,updated,published,title,authors,summary,comment,link,pdf_link,term
0,http://arxiv.org/abs/2304.13028v1,2023-04-25T17:59:03Z,2023-04-25T17:59:03Z,Unifying Emergent Hydrodynamics and Lindbladia...,"[Olumakinde Ogunnaike, Johannes Feldmeier, Jon...",We identify emergent hydrodynamics governing...,"6 + 4 pages, 4 figures",http://arxiv.org/abs/2304.13028v1,http://arxiv.org/pdf/2304.13028v1,cond-mat.str-el
1,http://arxiv.org/abs/2304.13017v1,2023-04-25T17:47:48Z,2023-04-25T17:47:48Z,DuETT: Dual Event Time Transformer for Electro...,"[Alex Labach, Aslesha Pokhrel, Xiao Shi Huang,...",Electronic health records (EHRs) recorded in...,None,http://arxiv.org/abs/2304.13017v1,http://arxiv.org/pdf/2304.13017v1,cs.LG
2,http://arxiv.org/abs/2212.04494v2,2023-04-25T17:14:44Z,2022-12-08T18:59:31Z,What it takes to solve the Hubble tension thro...,"[Nanoom Lee, Yacine Ali-Haïmoud, Nils Schönebe...",We construct data-driven solutions to the Hu...,"5+13 pages, 2+12 figures, version accepted for...",http://arxiv.org/abs/2212.04494v2,http://arxiv.org/pdf/2212.04494v2,astro-ph.CO
3,http://arxiv.org/abs/2304.12999v1,2023-04-25T17:13:57Z,2023-04-25T17:13:57Z,Imaging mechanism and contrast separation in l...,"[Boxiang Zhang, Zhiyong Zhang, Chuanhong Jin]",Polymer-sorted high-density carbon nanotube ...,"25 pages, 9 figures main text. 5 pages, 6 figu...",http://arxiv.org/abs/2304.12999v1,http://arxiv.org/pdf/2304.12999v1,physics.app-ph
4,http://arxiv.org/abs/2303.15436v2,2023-04-25T16:37:36Z,2023-03-27T17:58:46Z,Spectroscopy of hot-electron pair emission fro...,"[Jens Schulenborg, Jonathan D. Fletcher, Masay...",On-demand emission of individual electrons f...,11 pages plus appendix; submitted to Phys. Rev. B,http://arxiv.org/abs/2303.15436v2,http://arxiv.org/pdf/2303.15436v2,cond-mat.mes-hall
5,http://arxiv.org/abs/2304.12974v2,2023-04-26T11:14:35Z,2023-04-25T16:31:45Z,Study of the bipolar jet of the YSO Th 28 with...,"[S. Yu. Melnikov, P. A. Boley, N. S. Nikonova,...",$Context.$ The YSO Th 28 possesses a highly ...,"18 pages, 14 figures",http://arxiv.org/abs/2304.12974v2,http://arxiv.org/pdf/2304.12974v2,astro-ph.SR
6,http://arxiv.org/abs/2304.12266v2,2023-04-25T16:31:24Z,2023-04-24T17:06:51Z,Magnetic plateaus and jumps in a spin-1/2 ladd...,"[Sk Saniur Rahaman, Manoranjan Kumar, Shaon Sa...",We study a frustrated two-leg spin-1/2 ladde...,"12 pages, 7 figures",http://arxiv.org/abs/2304.12266v2,http://arxiv.org/pdf/2304.12266v2,cond-mat.str-el
7,http://arxiv.org/abs/2304.12965v1,2023-04-25T16:19:12Z,2023-04-25T16:19:12Z,Entanglement Transitions in Unitary Circuit Games,"[Raúl Morral-Yepes, Adam Smith, S. L. Sondhi, ...",Repeated projective measurements in unitary ...,"10 pages, 8 figures",http://arxiv.org/abs/2304.12965v1,http://arxiv.org/pdf/2304.12965v1,quant-ph
8,http://arxiv.org/abs/2304.12962v1,2023-04-25T16:14:00Z,2023-04-25T16:14:00Z,Nonequilibrium phononic first-order phase tran...,"[Mohsen Yarmohammadi, Marin Bukov, Michael H. ...",We study the direct laser drive of infrared-...,"7+6 pages, 4+4 figures",http://arxiv.org/abs/2304.12962v1,http://arxiv.org/pdf/2304.12962v1,cond-mat.str-el
9,http://arxiv.org/abs/2304.12957v1,2023-04-25T16:01:38Z,2023-04-25T16:01:38Z,Dynamical localization transition of string br...,"[Roberto Verdel, Guo-Yi Zhu, Markus Heyl]",The fission of a string connecting two charg...,"4.5+4 pages, 3+2 figures",http://arxiv.org/abs/2304.12957v1,http://arxiv.org/pdf/2304.12957v1,cond-mat.str-el


In [43]:
def get_num_figures(comment: str):
    if comment is None:
        return 0

    # regex pattern to match "n figures" or "n + m figures", or mixtures of both
    pattern = r"\d+\s*\+\s*\d+\s*figures|\d+\s*figures"
    matches = re.findall(pattern, comment)

    if len(matches) == 0:
        return 0
    else:
        num_figures = sum(sum(int(n) for n in re.findall(r"\d+", match))
                          for match in matches)
    return num_figures

In [57]:
def get_num_pages(comment: str):
    if comment is None:
        return 0

    # regex pattern to match "n pages" or "n + m pages", no mixtures
    pattern = r"\d+\s*\+\s*\d+\s*pages|\d+\s*pages"
    match = re.search(pattern, comment)
    
    if not match:
        return 0
    else:
        match = match.group()
        num_pages = sum(int(n) for n in re.findall(r"\d+", match))
    return num_pages

In [64]:
def get_number_eqs(summary: str):
    if summary is None:
        return 0
    
    pattern = r"\${1,2}([\s\S]+?)\${1,2}"
    matches = re.findall(pattern, summary)
    return len(matches)

In [65]:
df['num_figures'] = df['comment'].apply(get_num_figures)
df['num_pages'] = df['comment'].apply(get_num_pages)
df['num_eqs'] = df['summary'].apply(get_number_eqs)

In [66]:
df

,paper_id,updated,published,title,authors,summary,comment,link,pdf_link,term,num_figures,num_pages,num_eqs
0,http://arxiv.org/abs/2304.13028v1,2023-04-25T17:59:03Z,2023-04-25T17:59:03Z,Unifying Emergent Hydrodynamics and Lindbladia...,"[Olumakinde Ogunnaike, Johannes Feldmeier, Jon...",We identify emergent hydrodynamics governing...,"6 + 4 pages, 4 figures",http://arxiv.org/abs/2304.13028v1,http://arxiv.org/pdf/2304.13028v1,cond-mat.str-el,4,10,0
1,http://arxiv.org/abs/2304.13017v1,2023-04-25T17:47:48Z,2023-04-25T17:47:48Z,DuETT: Dual Event Time Transformer for Electro...,"[Alex Labach, Aslesha Pokhrel, Xiao Shi Huang,...",Electronic health records (EHRs) recorded in...,None,http://arxiv.org/abs/2304.13017v1,http://arxiv.org/pdf/2304.13017v1,cs.LG,0,0,0
2,http://arxiv.org/abs/2212.04494v2,2023-04-25T17:14:44Z,2022-12-08T18:59:31Z,What it takes to solve the Hubble tension thro...,"[Nanoom Lee, Yacine Ali-Haïmoud, Nils Schönebe...",We construct data-driven solutions to the Hu...,"5+13 pages, 2+12 figures, version accepted for...",http://arxiv.org/abs/2212.04494v2,http://arxiv.org/pdf/2212.04494v2,astro-ph.CO,14,18,2
3,http://arxiv.org/abs/2304.12999v1,2023-04-25T17:13:57Z,2023-04-25T17:13:57Z,Imaging mechanism and contrast separation in l...,"[Boxiang Zhang, Zhiyong Zhang, Chuanhong Jin]",Polymer-sorted high-density carbon nanotube ...,"25 pages, 9 figures main text. 5 pages, 6 figu...",http://arxiv.org/abs/2304.12999v1,http://arxiv.org/pdf/2304.12999v1,physics.app-ph,15,25,0
4,http://arxiv.org/abs/2303.15436v2,2023-04-25T16:37:36Z,2023-03-27T17:58:46Z,Spectroscopy of hot-electron pair emission fro...,"[Jens Schulenborg, Jonathan D. Fletcher, Masay...",On-demand emission of individual electrons f...,11 pages plus appendix; submitted to Phys. Rev. B,http://arxiv.org/abs/2303.15436v2,http://arxiv.org/pdf/2303.15436v2,cond-mat.mes-hall,0,11,0
5,http://arxiv.org/abs/2304.12974v2,2023-04-26T11:14:35Z,2023-04-25T16:31:45Z,Study of the bipolar jet of the YSO Th 28 with...,"[S. Yu. Melnikov, P. A. Boley, N. S. Nikonova,...",$Context.$ The YSO Th 28 possesses a highly ...,"18 pages, 14 figures",http://arxiv.org/abs/2304.12974v2,http://arxiv.org/pdf/2304.12974v2,astro-ph.SR,14,18,16
6,http://arxiv.org/abs/2304.12266v2,2023-04-25T16:31:24Z,2023-04-24T17:06:51Z,Magnetic plateaus and jumps in a spin-1/2 ladd...,"[Sk Saniur Rahaman, Manoranjan Kumar, Shaon Sa...",We study a frustrated two-leg spin-1/2 ladde...,"12 pages, 7 figures",http://arxiv.org/abs/2304.12266v2,http://arxiv.org/pdf/2304.12266v2,cond-mat.str-el,7,12,3
7,http://arxiv.org/abs/2304.12965v1,2023-04-25T16:19:12Z,2023-04-25T16:19:12Z,Entanglement Transitions in Unitary Circuit Games,"[Raúl Morral-Yepes, Adam Smith, S. L. Sondhi, ...",Repeated projective measurements in unitary ...,"10 pages, 8 figures",http://arxiv.org/abs/2304.12965v1,http://arxiv.org/pdf/2304.12965v1,quant-ph,8,10,1
8,http://arxiv.org/abs/2304.12962v1,2023-04-25T16:14:00Z,2023-04-25T16:14:00Z,Nonequilibrium phononic first-order phase tran...,"[Mohsen Yarmohammadi, Marin Bukov, Michael H. ...",We study the direct laser drive of infrared-...,"7+6 pages, 4+4 figures",http://arxiv.org/abs/2304.12962v1,http://arxiv.org/pdf/2304.12962v1,cond-mat.str-el,8,13,0
9,http://arxiv.org/abs/2304.12957v1,2023-04-25T16:01:38Z,2023-04-25T16:01:38Z,Dynamical localization transition of string br...,"[Roberto Verdel, Guo-Yi Zhu, Markus Heyl]",The fission of a string connecting two charg...,"4.5+4 pages, 3+2 figures",http://arxiv.org/abs/2304.12957v1,http://arxiv.org/pdf/2304.12957v1,cond-mat.str-el,5,9,0
